In [1]:
import pandas as pd
import numpy as np
import wrds
from pathlib import Path  

Process for each index/country:
1. Get list of index constituents from Compustat
2. Pass list of index constituents to CRSP data
3. Calculate Adj. Close etc.

In [2]:
# List of countries and currency codes to get data for:
data_ = pd.read_csv (f'C:/Users/Carla/Dropbox/Uni/10. Semester/Seminar Fin Econ/Data/countrycodes.csv')
countrycds = data_['ISO Country Code']
curcdds = data_['ISO Currency Code']
pars = zip(countrycds, curcdds)

# Connect to wrds
db = wrds.Connection(wrds_username='anfr18ab')

Loading library list...
Done


In [ ]:
for par in pars:
    try:        
        # Set country AND currencycode here
        countrycd = par[0]
        df_curcdd = par[1]

        junedate = '2000-01-01'

        # Get world index constituents data
        sql_wic = f"select fic, gvkey, iid, junedate from wrdsapps_windices.wcountryconstituents where fic = '{countrycd}' and junedate > '2000-01-01' LIMIT 10000"
        data_wic = db.raw_sql(sql_wic) # Chose limit here
        #print(f' Observations, wic data: {len(data_wic)}')

        data_wic["gvkey"]=data_wic["gvkey"].apply(str)
        filepath = Path(f'C:/Users/Carla/Dropbox/Uni/10. Semester/Seminar Fin Econ/Data/WIC_{countrycd}.csv') # Set name
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        data_wic.to_csv(filepath, index=False) # # Save WIC

        # Set up list of gvkeys to pass to CRSP query
        gvkeys_list = data_wic['gvkey']
        gvkeys_list.drop_duplicates()
        gvkeys = {}
        gvkeys["gvkeys"] = tuple(gvkeys_list)
        print(f' Index: {countrycd}, total companies in scope: {len(gvkeys_list.drop_duplicates())}')

        # Get CRSP primary issue ID
        data_crspiid = db.raw_sql("select gvkey, priusa, prirow "
                   "from crsp_a_ccm.comphead "
                   "where gvkey in %(gvkeys)s " # Chosing gvkeys here
                  " LIMIT 1", # Chose limit here
                    params=gvkeys)  # Pass list of gvkeys

        data_crspiid = data_crspiid.drop(columns='priusa') #, 'priusa', 'prirow', 'primaryissue']) # Dont need USA here
        #print(f' Observations, crspiid data: {len(data_crspiid)}')

        # Get CRSP stock data 
        data_crsp = db.raw_sql("select gvkey, iid, datadate, ajexdi, prccd, trfd, curcdd, conm, cshoc "#, priusa, prirow, currcd "
                   "from comp_global_daily.g_secd "
                   "where gvkey in %(gvkeys)s " # Chosing gvkeys here
                   "and datadate > '2000-01-01'" # Chose start date here
                    "and datadate < '2021-12-31'", # Chose end date here
                  #" LIMIT 10", # Chose limit here
                    params=gvkeys)  # Pass list of gvkeys
        print(f' Total daily observations, crsp data: {len(data_crsp)}')

        # Copy CRSP data and start formatting
        df = data_crsp.copy()

        # 1. Merge CRSP with IID primary issues so we can filter primary issue ID
        df = pd.merge(left=data_crsp, right=data_crspiid, left_on="gvkey", right_on="gvkey", how="left")
        df.prirow.fillna(df.iid, inplace=True) # fill missing values
        df["primaryissue"] = np.where(df.iid==df.prirow, 'True', 'False') # Creates Primary issue column
        df = df[df.primaryissue == 'True'] # If issue ID = primary issue, then keep, otherwise remove

        # 2. Calculate adjusted close = (prccd/ajexdi)*trfd and mktcap
        df["adjclose"] = (df.prccd/df.ajexdi)*df.trfd
        df["mktcap"] = df.adjclose*df.cshoc

        # 3. Format date as date
        df['date'] =  df['datadate']
        #df['Date'] =  pd.to_datetime(df['datadate'], format='%Y%m%d')

        # 4. Select based on currency code
        df = df[df.curcdd == df_curcdd]

        # 5. Drop unneeded columns
        df = df.drop(columns=['conm','iid', 'datadate', 'ajexdi', 'prccd', 'trfd', 'curcdd', 'prirow', 'primaryissue']) #, 'priusa', 'prirow', 'primaryissue'])
        df = df.dropna()

        # 6. Save country price data
        df["gvkey"]=df["gvkey"].apply(str)
        filepath = Path(f'C:/Users/Carla/Dropbox/Uni/10. Semester/Seminar Fin Econ/Data/{countrycd}.csv') # Set name
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        df.to_csv(filepath, index=False)
    except:
        print(f'Error at {countrycd}')
        continue
        


 Index: AUS, total companies in scope: 1825


In [ ]:
db.close()

# HELP


In [ ]:
# db.create_pgpass_file() # Creating password file - only for first time setup

#help(db.get_table)
#help(db.raw_sql)

# Finding libarires:
#sorted(db.list_libraries())
#db.list_tables(library="wrdsapps_windices")
#db.list_tables(library="comp_global_daily")

#db.describe_table(library="wrdsapps_windices", table="wcountryconstituents")
#db.describe_table(library="comp_global_daily", table="g_secd")

# Note libraries i need:
#wrdsapps_windices
#comp_global_daily
#crspa

# Tables i might need
#wcountryconstituents
#g_secd'
#g_funda
#ccm_lookup